In [17]:
import requests


def fetch_and_save_organization_ids_and_categories(base_url, params, output_file):
    with open(output_file, 'w') as file:
        page = 1
        total_ids = 0

        while True:
            params['page'] = page
            response = requests.get(base_url, params=params)
            if response.status_code != 200:
                print(f"Failed to fetch data for page {page}. Status code: {response.status_code}")
                break

            data = response.json()
            if not data:
                print("No more data to fetch.")
                break

            for org in data:
                id = str(org['id'])
                category = org.get('category', 'N/A')
                file.write(f"{id}, {category}\n")

            total_ids += len(data)
            print(f"Fetched {len(data)} records for page {page}. Total records so far: {total_ids}")

            page += 1

        print(f"Finished fetching. Total records fetched: {total_ids}")


base_url = "https://hcb.hackclub.com/api/v3/organizations"
params = {'per_page': 100, 'page': 1}
output_file = "orgs.txt"

fetch_and_save_organization_ids_and_categories(base_url, params, output_file)

Fetched 100 records for page 1. Total records so far: 100
Fetched 100 records for page 2. Total records so far: 200
Fetched 100 records for page 3. Total records so far: 300
Fetched 100 records for page 4. Total records so far: 400
Fetched 100 records for page 5. Total records so far: 500
Fetched 100 records for page 6. Total records so far: 600
Fetched 100 records for page 7. Total records so far: 700
Fetched 100 records for page 8. Total records so far: 800
Fetched 77 records for page 9. Total records so far: 877
No more data to fetch.
Finished fetching. Total records fetched: 877


In [11]:
# id: org_Yvguja
'''
curl --request GET \
  --url https://hcb.hackclub.com/api/v3/organizations/organization_id/transactions \
  --header 'Accept: application/json'
'''

org_id = "org_YVdu65"
data = requests.get(f"https://hcb.hackclub.com/api/v3/organizations/{org_id}/transactions", headers={'Accept': 'application/json'}).json()


[[{'id': 'tag_oB4TWj', 'object': 'tag', 'label': '💻Laptop'}]]

In [13]:
[x for x in data if x['tags']]

[{'id': 'txn_K9HNrK',
  'object': 'transaction',
  'href': 'https://hcb.hackclub.com/api/v3/transactions/txn_K9HNrK.json',
  'amount_cents': -154115,
  'memo': 'TRANSFER TO TRAVEL GRANT FOR IVOINE',
  'date': '2023-02-06',
  'type': 'transfer',
  'pending': False,
  'receipts': {'count': 0, 'missing': False},
  'comments': {'count': 0},
  'organization': {'id': 'org_YVdu65',
   'object': 'organization',
   'href': 'https://hcb.hackclub.com/api/v3/organizations/org_YVdu65.json'},
  'tags': [{'id': 'tag_oB4TWj', 'object': 'tag', 'label': '💻Laptop'}],
  'transfer': {'id': 'xfr_Kjsdd9',
   'object': 'transfer',
   'href': 'https://hcb.hackclub.com/api/v3/transfers/xfr_Kjsdd9.json'}}]

In [1]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed


def fetch_transactions(org_id, org_category):
    transactions = []
    page = 1
    while True:
        response = requests.get(
            f"https://hcb.hackclub.com/api/v3/organizations/{org_id}/transactions",
            headers={'Accept': 'application/json'},
            params={'per_page': 50, 'page': page}
        )
        data = response.json()
        if not data:
            break

        for transaction in data:
            transaction_record = {
                'amount_cents': transaction['amount_cents'],
                'memo': transaction['memo'],
                'date': transaction['date'],
                'type': transaction['type'],
                'tags': [x['label'] for x in transaction['tags']],
                'org_category': org_category,
                'org_id': org_id,
                'check_memo': transaction['check']['memo'] if transaction.get('check')  and transaction['check'].get('memo') else None,
                'donation_memo': transaction['donation']['memo'] if transaction.get('donation') and transaction['donation'].get('memo') else None,
                'invoice_memo': transaction['invoice']['memo'] if transaction.get('invoice') and transaction['invoice'].get('memo') else None,
                'transfer_memo': transaction['transfer']['memo'] if transaction.get('transfer') and transaction['transfer'].get('memo') else None,
            }
            transactions.append(transaction_record)

        print(f"Fetched page {page} for organization {org_id} ({org_category})")
        page += 1

    return transactions

In [2]:
with open("orgs.txt", 'r') as file:
        orgs = [line.strip().split(",") for line in file.readlines()]


In [3]:
all_transactions = []

with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_org = {executor.submit(fetch_transactions, org_id, org_category): (
        org_id, org_category) for org_id, org_category in orgs}
    for future in as_completed(future_to_org):
        org_id, org_category = future_to_org[future]
        try:
            transactions = future.result()
            all_transactions.extend(transactions)
        except Exception as exc:
            print(f'Organization {org_id} ({org_category}) generated an exception: {exc}')

df = pd.DataFrame(all_transactions)
df.to_csv('transactions2.csv', index=False)

Organization org_Yvguja ( high_school_hackathon) generated an exception: 'memo'
Organization org_Y36uWe ( high_school_hackathon) generated an exception: 'memo'
Organization org_MpJu4A ( high_school_hackathon) generated an exception: 'memo'
Organization org_YVdu65 ( hack_club_hq) generated an exception: 'memo'
Organization org_YdkuJ6 ( high_school_hackathon) generated an exception: 'memo'
Fetched page 1 for organization org_YKeuOB ( high_school_hackathon)
Organization org_aXDuR0 ( hack_club_hq) generated an exception: 'memo'
Organization org_MONuWk ( high_school_hackathon) generated an exception: 'memo'


KeyboardInterrupt: 